In [ ]:
#@markdown ### زمان تقریبی ۱۲ دقیقه
#@title نصب
print("--> Preparing Colab with CUDA 11.8...")
!apt-get update > /dev/null
!apt-get install -y --no-install-recommends \
    cuda-11-8 libcudnn8=8.6.0.163-1+cuda11.8 libcudnn8-dev=8.6.0.163-1+cuda11.8

!pip install onnxruntime-gpu

import os
os.environ["CUDA_PATH"] = "/usr/local/cuda"
os.environ["LD_LIBRARY_PATH"] += ":/usr/local/cuda/lib64"
os.environ["PATH"] += ":/usr/local/cuda/bin"
from IPython.display import clear_output
clear_output()

--> Preparing Colab with CUDA 11.8...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  cuda-cccl-11-8 cuda-command-line-tools-11-8 cuda-compiler-11-8
  cuda-cudart-11-8 cuda-cudart-dev-11-8 cuda-cuobjdump-11-8 cuda-cupti-11-8
  cuda-cupti-dev-11-8 cuda-cuxxfilt-11-8 cuda-demo-suite-11-8
  cuda-documentation-11-8 cuda-driver-dev-11-8 cuda-drivers cuda-drivers-580
  cuda-gdb-11-8 cuda-libraries-11-8 cuda-libraries-dev-11-8 cuda-memcheck-11-8
  cuda-nsight-11-8 cuda-nsight-compute-11-8 cuda-nsight-systems-11-8
  cuda-nvcc-11-8 cuda-nvdisasm-11-8 cuda-nvml-dev-11-8 cuda-nvprof-11-8
  cuda-nvprune-11-8 cuda-nvrtc-11-8 cuda-nvrtc-dev-11-8 cuda-nvtx-11-8
  cuda-nvvp-11-8 cuda-profiler-api-11-8 cuda-runtime

In [ ]:
#@title Clone
#@markdown ###حتما اجرا شود
print("--> Cloning your DeepFake repository...")
!git clone https://github.com/menchbaz/DeepFake
%cd DeepFake

print("--> Installing requirements...")
!pip install -r requirements.txt
!pip install keras --upgrade

import onnxruntime
print("\n--> Verifying GPU availability...")
print(f"Available Providers: {onnxruntime.get_available_providers()}")
!pip install customtkinter tkinterdnd2 opennsfw2 psutil
!pip install insightface==0.7.3
clear_output()
print("✅✅✅ Installation and setup complete! ✅✅✅")

In [ ]:

#@title آپلود عکس

import os
from IPython.display import display, Image
from google.colab import files
import shutil

# Directory name is now your repository name
directory = 'DeepFake'
base_path = os.path.join("/content", directory)
image_path = os.path.join(base_path, "source.jpg")

# Delete existing "source.jpg" file if it exists
if os.path.exists(image_path):
    os.remove(image_path)
    print("Deleted existing source.jpg file.")

# Allow user to upload image
uploaded = files.upload()

# Move and rename the uploaded file to "source.jpg"
for filename in uploaded.keys():
    shutil.move(filename, image_path)
    print(f"Moved and renamed {filename} to source.jpg")

# Preview uploaded image
if os.path.exists(image_path):
    display(Image(filename=image_path, width=500))
else:
    print("Image file not found.")

In [ ]:

#@title آپلود ویدیوی هدف
#@markdown - It is recommended to upload videos not longer than 30 seconds to speed up the process.

import os
import shutil
from IPython.display import display, Video, HTML
from google.colab import files
from base64 import b64encode

# Directory name is now your repository name
directory = 'DeepFake'
base_path = os.path.join("/content", directory)
video_path = os.path.join(base_path, "target.mp4")

# Ensure the directory exists
os.makedirs(base_path, exist_ok=True)

# Delete existing "target.mp4" file
if os.path.exists(video_path):
    os.remove(video_path)
    print("Deleted existing target.mp4 file.")

# Allow user to upload video file
uploaded = files.upload()

# Move and rename the uploaded file to "target.mp4"
for filename in uploaded.keys():
    shutil.move(filename, video_path)
    print(f"Moved and renamed {filename} to target.mp4")

# Show uploaded video
print("\nLoading Final Video Preview, Please Wait...")
if os.path.exists(video_path):
    mp4 = open(video_path, 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    display(HTML(f"""
    <video width=720 controls>
          <source src="{data_url}" type="video/mp4">
    </video>
    """))
else:
    print("Video file not found.")

In [ ]:

#@title اجرای دیپ فیک

face_selection_mode = "Swap Default Face (Fastest)" #@param ["Swap Default Face (Fastest)", "Swap ALL faces in video", "Swap a SPECIFIC face"]
specific_face_number = 0 #@param {type:"number"}

# --- Script ---
import os
from base64 import b64encode
from IPython.display import display, HTML

%cd "/content/DeepFake"

# Define file paths
output_video_path = "face_changed_video.mp4"

# Delete existing output file
if os.path.exists(output_video_path):
    os.remove(output_video_path)
    print("Deleted existing output video file.")

# Base command
command = (
    "python run.py -s 'source.jpg' -t 'target.mp4' -o 'face_changed_video.mp4' "
    "--keep-fps --output-video-quality 1 --execution-provider cuda --frame-processor face_swapper"
)

# Add face selection options based on user's choice
if "ALL faces" in face_selection_mode:
    command += " --many-faces"
elif "SPECIFIC face" in face_selection_mode:
    command += f" --reference-face-position {specific_face_number}"

# Execute the final command
print("\nExecuting command:")
print(command)
print("\n--- Starting Face Swapping Process... ---")
!{command}

# --- Display Final Video ---
print("\n--- Process Finished ---")
if os.path.exists(output_video_path):
  print("Loading Final Video, Please Wait...")
  mp4 = open(output_video_path,'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  display(HTML(f"""
  <video width=720 controls>
        <source src="{data_url}" type="video/mp4">
  </video>
  """))
else:
  print("⚠️ Output video not found. The process may have failed.")


image_url = 'https://huggingface.co/Toolsai/dubtest/resolve/main/newgolden.png'
youtube_channel_url = 'https://youtube.com/@aigolden'

html_code = f'''
<div style="text-align: center; border: 2px solid #e0e0e0; padding: 15px; border-radius: 12px; background-color: #f9f9f9; max-width: 350px; margin: auto;">
    <a href="{youtube_channel_url}" target="_blank" title="رفتن به کانال یوتیوب AIGOLDEN">
        <img src="{image_url}" alt="AIGOLDEN YouTube Channel" style="max-width: 100%; height: auto; border-radius: 8px;">
    </a>
    <p style="font-size: 16px; font-family: 'Vazir', sans-serif; margin-top: 15px; color: #333;">
        برای مشاهده آموزش‌های بیشتر، ما را در یوتیوب دنبال کنید.
    </p>
    <a href="{youtube_channel_url}" target="_blank" style="text-decoration: none; display: inline-block; background-color: #FF0000; color: white; padding: 10px 20px; border-radius: 8px; font-weight: bold; font-family: 'Vazir', sans-serif; margin-top: 10px;">
        🚀 دنبال کردن در یوتیوب
    </a>
</div>
'''
display(HTML(html_code))

In [ ]:

#@title دانلود
from google.colab import files
import os

video_path = "/content/DeepFake/face_changed_video.mp4"

if os.path.exists(video_path):
  print(f"Starting download for {video_path}...")
  files.download(video_path)
else:
  print(f"❌ Error: File not found at {video_path}")